In [1]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import torch.nn as nn

from neuralhydrology.utils.config import Config

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

import os

# MISC Function Test

In [2]:
from neuralhydrology.modelzoo.cfe_modules.dcfe_utils import get_dcfe_params
from neuralhydrology.datautils import utils

In [3]:
config_path = Path('/Users/ziyu/Library/CloudStorage/OneDrive-ColoradoSchoolofMines/Documents/College/ResearchStuff/NextGen/neuralhydrology-dcfe/examples/07-DCFE/2basinTest_dynamic.yml')
config = Config(config_path, dev_mode=True)
basins = utils.load_basin_file(getattr(config, "train_basin_file"))

testDF = get_dcfe_params(config)
testDF.loc['02177000']

depth                                                        tensor(2.)
bb                                                       tensor(1.8747)
satdk                                                    tensor(0.0007)
satpsi                                                   tensor(0.1889)
slop                                                     tensor(0.8340)
smcmax                                                   tensor(0.2987)
wltsmc                                                   tensor(0.0521)
D                                                            tensor(2.)
mult                                                         tensor(1.)
catchment_area_km2                                     tensor(111.1100)
refkdt                                                   tensor(0.8883)
max_gw_storage                                           tensor(0.2016)
expon                                                    tensor(1.8009)
Cgw                                                      tensor(

We expect the above to return 
depth                                                        tensor(2.)
bb                                                       tensor(1.8747)
satdk                                                    tensor(0.0007)
satpsi                                                   tensor(0.1889)
slop                                                     tensor(0.8340)
smcmax                                                   tensor(0.2987)
wltsmc                                                   tensor(0.0521)
D                                                            tensor(2.)
mult                                                         tensor(1.)
catchment_area_km2                                     tensor(111.1100)
refkdt                                                   tensor(0.8883)
max_gw_storage                                           tensor(0.2016)
expon                                                    tensor(1.8009)
Cgw                                                      tensor(0.0003)
alpha_fc                                                 tensor(0.3300)
K_nash                                                   tensor(0.0980)
K_lf                                                     tensor(0.8121)
nash_storage                                   [tensor(0.), tensor(0.)]
giuh_ordinates        [tensor(0.3300), tensor(0.2900), tensor(0.1900...
Name: 02177000, dtype: object

# Dynamic Daily

In [2]:
start_run(config_file=Path("2basinTest_dynamic.yml"), gpu=-1)

2025-12-01 13:28:03,102: Logging to /Users/ziyu/Library/CloudStorage/OneDrive-ColoradoSchoolofMines/Documents/College/ResearchStuff/NextGen/neuralhydrology-dcfe/examples/07-DCFE/runs/DevMultiBasin_Test_0112_132803/output.log initialized.
2025-12-01 13:28:03,102: ### Folder structure created at /Users/ziyu/Library/CloudStorage/OneDrive-ColoradoSchoolofMines/Documents/College/ResearchStuff/NextGen/neuralhydrology-dcfe/examples/07-DCFE/runs/DevMultiBasin_Test_0112_132803
2025-12-01 13:28:03,103: ### Run configurations for DevMultiBasin_Test
2025-12-01 13:28:03,103: experiment_name: DevMultiBasin_Test
2025-12-01 13:28:03,103: train_basin_file: 2_basin.txt
2025-12-01 13:28:03,104: validation_basin_file: 2_basin.txt
2025-12-01 13:28:03,104: test_basin_file: 2_basin.txt
2025-12-01 13:28:03,105: train_start_date: 2000-10-01 00:00:00
2025-12-01 13:28:03,105: train_end_date: 2014-09-30 00:00:00
2025-12-01 13:28:03,106: validation_start_date: 1990-10-01 00:00:00
2025-12-01 13:28:03,106: validatio

KeyError: 'static_conceptual_params'